![@mikegchambers](../../images/header.png)

# Convolutional Neural Networks

In this notebook, we explore Convolutional Neural Networks using TensorFlow and a custom Lego dataset.

![Lens](lens.png)

UPDATE: Select the `conda_tensorflow2_p310` kernel when prompted. 

In [ ]:
import tensorflow as tf
# tf.logging.set_verbosity(tf.logging.ERROR) # <- Update - Reomved this line as it's no longer compatible with the version of TF used.

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# The Data

Let's load out dataset.  Here we use Numpy to load in two datasets that have been created and saved previously using Numpy.  We have some images, and some labels.

In [ ]:
X = np.load('images_48.npy')
y = np.load('labels_48.npy')

Now we use scikit-learn to randomize the data, and split it into a training and a test dataset.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

The labels for the data are set to integer values.  Let's create a list with the official Lego part numbers. 

In [ ]:
class_names = ['lg_2412b', 'lg_3001', 'lg_3002', 'lg_3003', 'lg_3004', 'lg_3005', 'lg_3010', 'lg_3622', 'lg_3648', 'lg_3839']

Now, let's look at the data.  This time, we loop through the data to find the first occurance of each class.  This way we don't leave it to chance, and we have a referance of what each brick will look like.

In [ ]:
plt.figure(figsize=(20,10))
for b in range(10):
    for i in range(len(y_train)):
        if y_train[i] == b:
            plt.subplot(2,5,b+1)
            plt.xticks([])
            plt.yticks([])
            plt.imshow(X_train[i], cmap=plt.cm.binary)
            plt.xlabel("{} ({})".format(class_names[y_train[i]], b))
            break
plt.show()

# The Model

In this lesson we use a slightly different notation to create a Keras Sequential network.  Either method will work, this is not a CNN thing.  

The difference is that we are now including convolutional layers, and max pooling layers.

You can, and should experiment, by adding layers, removing layers and changing the size of layers to see what effect it has on the model.

In [ ]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(48, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(64, (9, 9), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

Next, we compile the model with our usual parameters.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Let's train... (again we store the output so we can graph it later)

In [ ]:
e = model.fit(X_train.reshape(-1, 48, 48,1), y_train, epochs=5)

# Training Summary Graph

Let's plot accuracy and loss over the epochs.  Does it look promising?

In [ ]:
plt.plot(e.history['accuracy']) # <- Note minor change here from `acc` to `accuracy`. 
plt.plot(e.history['loss'])
plt.show()

# Evaluation

Now let's ask the model to evaluate itself using the remaining test data that it's not seen before.

How well does it do?  Does the accuracy match the training accuracy?

In [ ]:
test_loss, test_acc = model.evaluate(X_test.reshape(-1, 48, 48, 1), y_test)
print('Test accuracy:', test_acc)

# Mass Prediction Results

We're going to display a graph of the first set of predictions.  But we're going to be a little more sophisticated this time and display the confidence of the prediction as well.

To do that we are first going to define a couple of functions to help with the display.

In [ ]:
# This function will display the image 
# and color the label in ref to a correct prediction:

def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap=plt.cm.binary)
    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'green'
    else:
        color = 'red'
    # Print a label with 'predicted class', 'probability %', 'actual class'
    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

# This function will display the prediction results in a bar graph:

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.xticks(range(10))
    plt.yticks([])
    plot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)
    plot[predicted_label].set_color('red')
    plot[true_label].set_color('green')

Lets get predictions for all of our test data.

In [ ]:
predictions = model.predict(X_test.reshape(-1, 48, 48, 1))

And now display the grid of results, using our defined display functions

In [ ]:
num_rows = 4
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(15, 10))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, y_test, X_test)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, y_test)
plt.show()

# Test with a real photo

Ok, so the model looks to be doing OK with test render images.  What about real-world images.

Let's load some more libraries and have a go.  These extra libraries are mostly about pre-processing the image to be the same size as our training data.

In [ ]:
import cv2
import PIL
import PIL.ImageOps
from PIL import Image, ImageEnhance
from skimage.util import random_noise

Now lets load the photo:

In [ ]:
## Load the data from disk:

img = cv2.imread('lg_3622.jpg')
# img = cv2.imread('lg_3004.jpg')
# img = cv2.imread('lg_3002.jpg')

## Convert the image to a Pillow image:

img = Image.fromarray(img)


## Quickly plot the original image so we can see what it looks like

plt.imshow(img)
plt.xticks([])
plt.yticks([])
plt.show()

While most of this code is optional, still run this cell as there is at least one line there!

In [ ]:
## This section can be un-commented as required to perform image enhancement:

# enhancer = ImageEnhance.Contrast(img)
# factor = 0.3
# img = enhancer.enhance(factor)

# enhancer = ImageEnhance.Brightness(img)
# factor = 1.4
# img = enhancer.enhance(factor)

# enhancer = ImageEnhance.Sharpness(img)
# factor = 5
# img = enhancer.enhance(factor)

img = np.asarray(img)

## Adding noise to the iamge can sometimes help 
## (but you would need to have added noise to the training data too)

# img = random_noise(img, mode='gaussian', var=0.001)
# img = np.array(255*img, dtype = 'uint8')

Now we convert the image to grayscale, change the size (the photo needed to be square to work well).

In [ ]:
img = Image.fromarray(img)

img = PIL.ImageOps.invert(img)
img = img.convert('L')
img.thumbnail((48, 48))

img = np.array(img)
img = img.astype('float32')
data = img/255.0

What does our processed image look like?

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(5,5,1)
plt.imshow(data, cmap=plt.cm.binary)
plt.xticks([])
plt.yticks([])
plt.show()

Ok, let's pass the processed image to our model and see what it thinks.

In [ ]:
 p = model.predict(data.reshape(1, 48, 48, 1)) # <- UPDATE: Minor update to the reshape here.
pp = p.argmax()
print("This is a photo of brick {} ({})".format(class_names[pp], pp))

You can try with your own images of Lego bricks (that the model is trained on).  Do you have any success?  The brick needs to be lit in the same way, and be generally the same size in the frame.